In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import libraries
import librosa
import librosa.display
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
from tensorflow.keras.models import load_model

In [3]:
# Load  CSV into a pandas DataFrame
features = pd.read_csv("/content/drive/Shareddrives/Machine Learning Group Project/encoded_features.csv")

In [4]:
features.head()

,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,chroma_cens.9,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
0,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,1.341231,...,0.054125,0.012226,0.012111,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448
1,1.888963,0.760539,0.345297,2.295201,1.654031,0.067592,1.366848,1.054094,0.108103,0.619185,...,0.063831,0.014212,0.017740,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330
2,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,1.038546,...,0.040730,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861
3,3.702245,-0.291193,2.196742,-0.234449,1.367364,0.998411,1.770694,1.604566,0.521217,1.982386,...,0.074358,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800
4,-0.193837,-0.198527,0.201546,0.258556,0.775204,0.084794,-0.289294,-0.816410,0.043851,-0.804761,...,0.095003,0.022492,0.021355,16.669037,0.469727,0.047225,0.040039,0.000977,3.189831,0.030993


In [5]:
# Drop rows with missing values
features = features.dropna()

# Extract features (X) and target variable (y), if applicable
X = features.values

# Normalize the data using Min-Max scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

features.shape

(106574, 518)

In [6]:
# Load the pre-trained autoencoder model
autoencoder = load_model('/content/drive/Shareddrives/Machine Learning Group Project/Features_AutoEncoder.h5')

In [12]:
# Function to extract audio features using librosa
def extract_audio_features(file_path, num_features=518):
    y, sr = librosa.load(file_path)

    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)

    # Take the mean along the time axis for each feature
    chroma_stft_mean = np.mean(chroma_stft, axis=1)
    mfcc_mean = np.mean(mfcc, axis=1)
    rmse_mean = np.mean(rmse, axis=1)

    # Keep only the specified number of features
    return np.concatenate((chroma_stft_mean[:num_features], mfcc_mean[:num_features], rmse_mean[:num_features]))


In [13]:
audio_features = extract_audio_features("/content/drive/Shareddrives/Machine Learning Group Project/Songs for Testing/twenty one pilots - Stressed Out (Audio).mp3")

ValueError: ignored

In [11]:
# Reshape audio features to (1, num_features)
audio_features = audio_features.reshape(1, -1)

# Ensure the reshaped audio features have the correct number of features
if audio_features.shape[1] != 518:
    raise ValueError(f"Expected 518 features, but got {audio_features.shape[1]} features.")

# Normalize the features using the scaler fitted with the training data
audio_features_scaled = scaler.transform(audio_features)

# Use the autoencoder to obtain the encoded representation
encoded_audio = autoencoder.predict(audio_features_scaled)


ValueError: ignored